<a href="https://colab.research.google.com/github/Redislabs-Solution-Architects/Redis-Workshops/blob/main/05-LangChain_Redis/05.12_LangChain_RedisCachedEmbeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Redis CacheBackedEmbeddings

![Redis](https://redis.com/wp-content/themes/wpx/assets/images/logo-redis.svg?auto=webp&quality=85,75&width=120)

This notebook goes over how to use Redis to cache the embeddings.

### Install Dependencies


In [1]:
!pip install -q langchain_openai langchain_community langchain redis unstructured


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.0/817.0 kB 28.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.3/250.3 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 44.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 246.4/246.4 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 41.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.2/62.2 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 421.5/421.5 kB 35.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.7/274.7 kB 25.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 55.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 51.7

## Initialize OpenAI

You need to supply the OpenAI API key (starts with `sk-...`) when prompted. You can find your API key at https://platform.openai.com/account/api-keys

In [2]:
import os
import getpass

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI Key: ")

OpenAI Key: ··········


### Install Redis Stack

Redis will be used as cacehd embedding store store for LangChain. Instead of using in-notebook Redis Stack https://redis.io/docs/getting-started/install-stack/ you can provision your own free instance of Redis in the cloud. Get your own Free Redis Cloud instance at https://redis.com/try-free/

In [3]:
%%sh
curl -fsSL https://packages.redis.io/gpg | sudo gpg --dearmor -o /usr/share/keyrings/redis-archive-keyring.gpg
echo "deb [signed-by=/usr/share/keyrings/redis-archive-keyring.gpg] https://packages.redis.io/deb $(lsb_release -cs) main" | sudo tee /etc/apt/sources.list.d/redis.list
sudo apt-get update  > /dev/null 2>&1
sudo apt-get install redis-stack-server  > /dev/null 2>&1
redis-stack-server --daemonize yes

deb [signed-by=/usr/share/keyrings/redis-archive-keyring.gpg] https://packages.redis.io/deb jammy main
Starting redis-stack-server, database path /var/lib/redis-stack


### Connect to Redis

By default this notebook would connect to the local instance of Redis Stack. If you have your own Redis Cloud instance - replace REDIS_PASSWORD, REDIS_HOST and REDIS_PORT values with your own.

In [4]:
import os


REDIS_HOST = os.getenv("REDIS_HOST", "localhost")
REDIS_PORT = os.getenv("REDIS_PORT", "6379")
REDIS_PASSWORD = os.getenv("REDIS_PASSWORD", "")
#Replace values above with your own if using Redis Cloud instance
#REDIS_HOST="redis-18374.c253.us-central1-1.gce.cloud.redislabs.com"
#REDIS_PORT=18374
#REDIS_PASSWORD="1TNxTEdYRDgIDKM2gDfasupCADXXXX"

#shortcut for redis-cli $REDIS_CONN command
# If SSL is enabled on the endpoint add --tls
if REDIS_PASSWORD!="":
  os.environ["REDIS_CONN"]=f"-h {REDIS_HOST} -p {REDIS_PORT} -a {REDIS_PASSWORD} --no-auth-warning"
else:
  os.environ["REDIS_CONN"]=f"-h {REDIS_HOST} -p {REDIS_PORT}"

# If SSL is enabled on the endpoint, use rediss:// as the URL prefix
REDIS_URL = f"redis://:{REDIS_PASSWORD}@{REDIS_HOST}:{REDIS_PORT}"

In [5]:
#test Redis connection
!redis-cli $REDIS_CONN PING

PONG


## Cached Embeddings

To initialize cached embeddings you need:
- Embedding object - OpenAIEmbeddings used here
- Store (key/value store - here we are using RedisStore backend)
- CacheBackedEmbeddings that assembles these together

In [6]:

from langchain_openai import OpenAIEmbeddings
from langchain.embeddings import CacheBackedEmbeddings
from langchain.storage import RedisStore

store = RedisStore(redis_url=REDIS_URL)

embedder = OpenAIEmbeddings()

cached_embedder = CacheBackedEmbeddings.from_bytes_store(
    embedder, store, namespace=embedder.model
)

In [7]:
sentences = [
        "Hi there!",
        "Oh, hello!",
        "What's your name?",
        "My friends call me World",
        "Hello World!"
        ]

In [8]:
# First - let's make sure there are no cached embeddings
list(store.yield_keys())
# you can also play around with the store object itself
#store.mset([("k1", b"v1"), ("k2", b"v2")])
#print(store.mget(["k1", "k2"]))

[]

In [9]:
%%time
# first run would invoke OpenAI embedding API
embeddings = cached_embedder.embed_documents(sentences)

CPU times: user 609 ms, sys: 47.4 ms, total: 656 ms
Wall time: 1.57 s


In [10]:
# now you should see five cached embeddings
list(store.yield_keys())

['text-embedding-ada-002b57fdad0-9cfa-5bac-9ffe-679b9e3a14a6',
 'text-embedding-ada-0027afa8587-1536-5b7c-8705-aaca3004e913',
 'text-embedding-ada-002f4641f10-cc90-5791-b0f3-b2a1555dac8f',
 'text-embedding-ada-002f26668b8-30e7-53ab-8662-6ec30ec795c9',
 'text-embedding-ada-002edef130a-43ed-57ff-8f52-b1c7ba00e8fb']

In [11]:
%%time
# second, cached run should be significantly faster
embeddings = cached_embedder.embed_documents(sentences)

CPU times: user 6.89 ms, sys: 0 ns, total: 6.89 ms
Wall time: 7.76 ms


### Cached embeddings with the Vectore Store

In [13]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import UnstructuredURLLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores.redis import Redis
import redis

# Add your own URLs here
urls = [
    "https://raw.githubusercontent.com/hwchase17/chat-your-data/master/state_of_the_union.txt"
]
loader = UnstructuredURLLoader(urls=urls)
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20, add_start_index = True)
texts = text_splitter.split_documents(documents)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [14]:
%%time
db1 = Redis.from_documents(
    texts,
    cached_embedder,
    redis_url=REDIS_URL,
    index_name="db1",
)

CPU times: user 393 ms, sys: 23 ms, total: 416 ms
Wall time: 836 ms


In [15]:
%%time
db2 = Redis.from_documents(
    texts,
    cached_embedder,
    redis_url=REDIS_URL,
    index_name="db2",
)

CPU times: user 85.7 ms, sys: 6.96 ms, total: 92.7 ms
Wall time: 114 ms


In [18]:
db2.similarity_search("International affairs")

[Document(page_content='It matters. American diplomacy matters. American resolve matters.\n\nPutin’s latest attack on Ukraine was premeditated and unprovoked.\n\nHe rejected repeated efforts at diplomacy.\n\nHe thought the West and NATO wouldn’t respond. And he thought he could divide us at home. Putin was wrong. We were ready. Here is what we did.\n\nWe prepared extensively and carefully.\n\nWe spent months building a coalition of other freedom-loving nations from Europe and the Americas to Asia and Africa to confront Putin.', metadata={'id': 'doc:db2:cfe3e0633051477e839a0b8f71d66b1c', 'source': 'https://raw.githubusercontent.com/hwchase17/chat-your-data/master/state_of_the_union.txt', 'start_index': '1847'}),
 Document(page_content='Groups of citizens blocking tanks with their bodies. Everyone from students to retirees teachers turned soldiers defending their homeland.\n\nIn this struggle as President Zelenskyy said in his speech to the European Parliament “Light will win over darkne

### Limitations

As of Langchain 0.1.9 embed_query method does not support caching for `embed_query` method.

In [20]:
cached_embedder.embed_query("hello world!")[:4]

[-0.007731702076730168,
 -0.005557568204855508,
 -0.016167470972066465,
 -0.033404081619237316]

### Redis storage for Vector DB and Embeddings cache

Finally let's look under the Redis covers. You should see keys like `text-embedding-ada-00255c1930e-bc0b-550d-a284-030f8cbfd05a` that are cached embeddings as well as `doc:db2:a0385cc570624e5d9e03eef8eb7f3d63` containing the Vector Database content

In [22]:
!redis-cli $REDIS_CONN keys "*"
!redis-cli $REDIS_CONN get "text-embedding-ada-00241c258d4-d253-5b09-813a-dc238f8e8ba7"
!redis-cli $REDIS_CONN hgetall "doc:db2:045063dc227c42d1be9ef090c408607c"

  1) "text-embedding-ada-002ae37faa6-caea-5d9d-b51a-06bf2b054500"
  2) "doc:db2:1ab6f4d6b7d34e33a4deab925b349ca0"
  3) "doc:db1:a9ce27bff6b1455c975d8190d49583fc"
  4) "text-embedding-ada-0025823e9b9-7f59-58de-9541-0c9b80eff57d"
  5) "doc:db2:045063dc227c42d1be9ef090c408607c"
  6) "doc:db2:d9c5e5a71c9147138daa88034b4d0f25"
  7) "text-embedding-ada-0023d1e0114-6569-5c12-bb44-8eda6ab6e7d5"
  8) "text-embedding-ada-00287cedfdf-aaab-53f2-8d6c-e42c899d51f9"
  9) "text-embedding-ada-00289ff6f98-c2a1-5534-97ee-163461304d4f"
 10) "text-embedding-ada-002bcaa4df1-54a7-5d11-afb6-ecb1cb76a747"
 11) "doc:db2:53d5e895c8a04c93825755e0c437dbcf"
 12) "doc:db2:ff2cb619c72843bab2b535654a1f9dfb"
 13) "doc:db1:e6e3ff633c3d4a0cadcbf6c407ffbf16"
 14) "text-embedding-ada-002f7f7dd89-dac6-573d-8ddd-126926320f18"
 15) "text-embedding-ada-0021b99c35b-0bd9-5cd6-9bba-451f85376112"
 16) "doc:db1:018ad68f46174734b73310af942b159e"
 17) "doc:db2:05dad6110ce84a288886d98e06651439"
 18) "doc:db2:77fabd5f009a4003a3faaa2323